#MOunt G Drive and Loading Data Sets

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir("/content/drive/MyDrive/Omdena Sri Lanka Dengue Fever/3. Data Pre-Processing/final_pre_processed")

MessageError: Error: credential propagation was unsuccessful

##Load Datasets  

In [ ]:
import pandas as pd
og_data=pd.read_csv(r"/content/drive/MyDrive/Omdena Sri Lanka Dengue Fever/3. Data Pre-Processing/final_pre_processed/Updated_Sri_lanka_dengue_cases_weather_weekly_2007_2024_.csv")
data=og_data.copy()
print("data_sets_created_sucessfully")

In [ ]:
#checking Data set
data.head()

#Importing All libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.dates as mdates
import lightgbm as lgb
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
# Suppress a specific warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Suppress all warnings
warnings.filterwarnings("ignore")


#EDA of Datasets

##Plot Number of Cases by District

In [ ]:
#plotting number of cases
import matplotlib.pyplot as plt

# Group by District and sum the Number of Cases
district_cases = data.groupby('District')['Number_of_Cases'].sum().reset_index()

# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.bar(district_cases['District'], district_cases['Number_of_Cases'])
plt.xlabel('Districts')
plt.ylabel('Number of Cases')
plt.title('Number of Cases by Districts')
plt.xticks(rotation=90)
plt.show()


### **Notes :**

## checking Multivariable correlation of all varaiables in all districts with 5 Types of Correlation methods (Spearman,Pearson,Kendall Tau,Partial Correlation,Distance Correlation )

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr, kendalltau
from pingouin import partial_corr
from scipy.spatial.distance import correlation as distance_correlation # Import distance correlation

def analyze_correlations(data, district_name):
    # Filter data for the specified district
    district_data = data[data['District'] == district_name]

    # Drop non-numeric columns and specified columns
    columns_to_drop = ['Month', 'Year', 'Week', 'Weather Code', 'Total Precipitation (mm)']
    district_data_numeric = district_data.drop(columns=columns_to_drop).select_dtypes(include=[np.number])

    # Calculate Spearman's Rank Correlation
    spearman_corr = district_data_numeric.corr(method='spearman')

    # Calculate Pearson's Correlation
    pearson_corr = district_data_numeric.corr(method='pearson')

    # Calculate Kendall's Tau Correlation
    kendall_corr = district_data_numeric.corr(method='kendall')

    # Calculate Partial Correlation
    partial_corr_matrix = pd.DataFrame(index=district_data_numeric.columns, columns=district_data_numeric.columns)
    for col1 in district_data_numeric.columns:
        for col2 in district_data_numeric.columns:
            if col1 != col2:
                partial_corr_matrix.loc[col1, col2] = partial_corr(data=district_data_numeric, x=col1, y=col2, covar=[col for col in district_data_numeric.columns if col not in [col1, col2]]).iloc[0]['r']
            else:
                partial_corr_matrix.loc[col1, col2] = 1.0

    # Calculate Distance Correlation
    distance_corr_matrix = district_data_numeric.apply(lambda x: district_data_numeric.apply(lambda y: distance_correlation(x, y)))

    # Assess the correlation of other variables with respect to "Number_of_Cases" column for all methods
    spearman_number_of_cases_corr = spearman_corr['Number_of_Cases'].drop('Number_of_Cases')
    pearson_number_of_cases_corr = pearson_corr['Number_of_Cases'].drop('Number_of_Cases')
    kendall_number_of_cases_corr = kendall_corr['Number_of_Cases'].drop('Number_of_Cases')
    partial_number_of_cases_corr = partial_corr_matrix['Number_of_Cases'].drop('Number_of_Cases')
    distance_number_of_cases_corr = distance_corr_matrix['Number_of_Cases'].drop('Number_of_Cases')

    # Plotting the heatmap for Spearman's Rank Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0)
    plt.title(f'Spearman Rank Correlation Heatmap for {district_name} District')
    plt.show()

    # Plotting the heatmap for Pearson's Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', center=0)
    plt.title(f'Pearson Correlation Heatmap for {district_name} District')
    plt.show()

    # Plotting the heatmap for Kendall's Tau Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(kendall_corr, annot=True, cmap='coolwarm', center=0)
    plt.title(f'Kendall Tau Correlation Heatmap for {district_name} District')
    plt.show()

    # Plotting the heatmap for Partial Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(partial_corr_matrix.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title(f'Partial Correlation Heatmap for {district_name} District')
    plt.show()

    # Plotting the heatmap for Distance Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(distance_corr_matrix.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title(f'Distance Correlation Heatmap for {district_name} District')
    plt.show()

    # Print the correlation of other variables with respect to "Number_of_Cases" for all methods
    print(f"Spearman Correlation of other variables with respect to 'Number_of_Cases' in {district_name} District:")
    print(spearman_number_of_cases_corr)
    print(f"\nPearson Correlation of other variables with respect to 'Number_of_Cases' in {district_name} District:")
    print(pearson_number_of_cases_corr)
    print(f"\nKendall Correlation of other variables with respect to 'Number_of_Cases' in {district_name} District:")
    print(kendall_number_of_cases_corr)
    print(f"\nPartial Correlation of other variables with respect to 'Number_of_Cases' in {district_name} District:")
    print(partial_number_of_cases_corr)
    print(f"\nDistance Correlation of other variables with respect to 'Number_of_Cases' in {district_name} District:")
    print(distance_number_of_cases_corr)


    # Create a DataFrame to compare the correlations
    comparison_df = pd.DataFrame({
        'Spearman': spearman_number_of_cases_corr,
        'Pearson': pearson_number_of_cases_corr,
        'Kendall': kendall_number_of_cases_corr,
        'Partial': partial_number_of_cases_corr,
        'Distance': distance_number_of_cases_corr
    })

    # Plotting the comparison chart
    comparison_df.plot(kind='bar', figsize=(12, 8))
    plt.title(f'Comparison of Spearman, Pearson, Kendall, Partial, and Distance Correlations with Number_of_Cases in {district_name} District')
    plt.xlabel('Variables')
    plt.ylabel('Correlation')
    plt.show()
# Apply the function to all unique districts
unique_districts = data['District'].unique()
for district in unique_districts:
    analyze_correlations(data, district)



## Insights and Reports of EDA on  District wise Correlation Charts





In [ ]:
# Generate a detailed insightful report of the output of the correlation analysis

def generate_insightful_report(data):
    unique_districts = data['District'].unique()
    report = []

    for district in unique_districts:
        district_data = data[data['District'] == district]
        columns_to_drop = ['District', 'Month', 'Year', 'Week', 'Weather Code', 'Total Precipitation (mm)']
        district_data_numeric = district_data.drop(columns=columns_to_drop).select_dtypes(include=[np.number])

        # Calculate correlations
        spearman_corr = district_data_numeric.corr(method='spearman')['Number_of_Cases']
        pearson_corr = district_data_numeric.corr(method='pearson')['Number_of_Cases']
        kendall_corr = district_data_numeric.corr(method='kendall')['Number_of_Cases']

        partial_corr_series = pd.Series(index=district_data_numeric.columns)
        for col in district_data_numeric.columns:
            if col != 'Number_of_Cases':
                partial_corr_series[col] = partial_corr(data=district_data_numeric, x='Number_of_Cases', y=col, covar=[c for c in district_data_numeric.columns if c not in ['Number_of_Cases', col]]).iloc[0]['r']
            else:
                partial_corr_series[col] = 1.0

        distance_corr = district_data_numeric.apply(lambda x: distance_correlation(district_data_numeric['Number_of_Cases'], x))

        # Create a DataFrame to compare the correlations
        comparison_df = pd.DataFrame({
            'Spearman': spearman_corr.drop('Number_of_Cases'),
            'Pearson': pearson_corr.drop('Number_of_Cases'),
            'Kendall': kendall_corr.drop('Number_of_Cases'),
            'Partial': partial_corr_series.drop('Number_of_Cases'),
            'Distance': distance_corr.drop('Number_of_Cases')
        })

        # Generate report for the district
        report.append(f"Correlation Analysis Report for {district} District")
        report.append("=" * 50)
        report.append(f"Top 5 Variables with Highest Spearman Correlation with 'Number_of_Cases':\n{spearman_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Pearson Correlation with 'Number_of_Cases':\n{pearson_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Kendall Correlation with 'Number_of_Cases':\n{kendall_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Partial Correlation with 'Number_of_Cases':\n{partial_corr_series.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Distance Correlation with 'Number_of_Cases':\n{distance_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append("\n")

        # Detailed insights
        report.append(f"Detailed Insights for {district} District:")
        report.append(f"The variables with the highest Spearman correlation with 'Number_of_Cases' indicate a monotonic relationship. This means that as one variable increases, the other variable tends to increase or decrease consistently.")
        report.append(f"The Pearson correlation highlights the linear relationships between 'Number_of_Cases' and other variables. High Pearson correlation values suggest that changes in these variables are linearly associated with changes in 'Number_of_Cases'.")
        report.append(f"Kendall correlation, similar to Spearman, measures the ordinal association between variables. High Kendall correlation values suggest a strong ordinal relationship.")
        report.append(f"Partial correlation values show the relationship between 'Number_of_Cases' and other variables while controlling for the effect of other variables. This helps in understanding the direct influence of each variable on 'Number_of_Cases'.")
        report.append(f"Distance correlation captures both linear and non-linear associations. High distance correlation values indicate strong associations, regardless of the nature of the relationship.")
        report.append("\n")

    # Print the report
    for line in report:
        print(line)

# Generate the report for all districts
generate_insightful_report(data)


##Checking the correlation of "Number of Cases" varaible with other variables in all district through heat map. also selelcting top   correlation varaible of feature selection of X axis for better training

In [ ]:
def calculate_and_plot_correlations(data):
    # Initialize dictionaries to store correlations for each district
    spearman_corr_dict = {}
    pearson_corr_dict = {}
    kendall_corr_dict = {}
    partial_corr_dict = {}
    distance_corr_dict = {}

    unique_districts = data['District'].unique()

    for district in unique_districts:
        district_data = data[data['District'] == district]
        columns_to_drop = ['District', 'Month', 'Year', 'Week', 'Weather Code', 'Total Precipitation (mm)']
        district_data_numeric = district_data.drop(columns=columns_to_drop).select_dtypes(include=[np.number])

        # Calculate correlations
        spearman_corr = district_data_numeric.corr(method='spearman')['Number_of_Cases']
        pearson_corr = district_data_numeric.corr(method='pearson')['Number_of_Cases']
        kendall_corr = district_data_numeric.corr(method='kendall')['Number_of_Cases']

        partial_corr_series = pd.Series(index=district_data_numeric.columns)
        for col in district_data_numeric.columns:
            if col != 'Number_of_Cases':
                partial_corr_series[col] = partial_corr(data=district_data_numeric, x='Number_of_Cases', y=col, covar=[c for c in district_data_numeric.columns if c not in ['Number_of_Cases', col]]).iloc[0]['r']
            else:
                partial_corr_series[col] = 1.0

        distance_corr = district_data_numeric.apply(lambda x: distance_correlation(district_data_numeric['Number_of_Cases'], x))

        # Store correlations in dictionaries
        spearman_corr_dict[district] = spearman_corr.drop('Number_of_Cases')
        pearson_corr_dict[district] = pearson_corr.drop('Number_of_Cases')
        kendall_corr_dict[district] = kendall_corr.drop('Number_of_Cases')
        partial_corr_dict[district] = partial_corr_series.drop('Number_of_Cases')
        distance_corr_dict[district] = distance_corr.drop('Number_of_Cases')

    # Convert dictionaries to DataFrames
    spearman_corr_df = pd.DataFrame(spearman_corr_dict)
    pearson_corr_df = pd.DataFrame(pearson_corr_dict)
    kendall_corr_df = pd.DataFrame(kendall_corr_dict)
    partial_corr_df = pd.DataFrame(partial_corr_dict)
    distance_corr_df = pd.DataFrame(distance_corr_dict)

    # Select top 5 variables based on absolute correlation values for each method
    top_variables = {
        'Spearman': spearman_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Pearson': pearson_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Kendall': kendall_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Partial': partial_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Distance': distance_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist()
    }

    print("Top 5 variables for feature engineering based on correlation methods:")
    for method, variables in top_variables.items():
        print(f"{method}: {variables}")

    # Plotting the heatmap for Spearman's Rank Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(spearman_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Spearman Rank Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Pearson's Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(pearson_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Pearson Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Kendall's Tau Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(kendall_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Kendall Tau Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Partial Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(partial_corr_df.T.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title('Partial Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Distance Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(distance_corr_df.T.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title('Distance Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Prepare data for the comparison line chart for top 5 variables
    top_variables_data = []
    for method, variables in top_variables.items():
        for variable in variables:
            top_variables_data.append({
                'Method': method,
                'Variable': variable,
                'Correlation': spearman_corr_df.loc[variable].mean() if method == 'Spearman' else
                               pearson_corr_df.loc[variable].mean() if method == 'Pearson' else
                               kendall_corr_df.loc[variable].mean() if method == 'Kendall' else
                               partial_corr_df.loc[variable].mean() if method == 'Partial' else
                               distance_corr_df.loc[variable].mean()
            })

    top_variables_df = pd.DataFrame(top_variables_data)

    # Plotting the comparison line chart for top 5 variables
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=top_variables_df, x='Variable', y='Correlation', hue='Method', marker='o')
    plt.title('Comparison of Top 5 Variables for Spearman, Pearson, Kendall, Partial, and Distance Correlations with respect to Number_of_Cases varaibale')
    plt.xlabel('Variables')
    plt.ylabel('Correlation')
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Call the function to calculate and plot correlations for all districts
calculate_and_plot_correlations(data)


## Report of EDA on Heat Map of All Districts

In [ ]:
def calculate_and_plot_correlations(data):
    # Initialize dictionaries to store correlations for each district
    spearman_corr_dict = {}
    pearson_corr_dict = {}
    kendall_corr_dict = {}
    partial_corr_dict = {}
    distance_corr_dict = {}

    unique_districts = data['District'].unique()

    for district in unique_districts:
        district_data = data[data['District'] == district]
        columns_to_drop = ['District', 'Month', 'Year', 'Week', 'Weather Code', 'Total Precipitation (mm)']
        district_data_numeric = district_data.drop(columns=columns_to_drop).select_dtypes(include=[np.number])

        # Calculate correlations
        spearman_corr = district_data_numeric.corr(method='spearman')['Number_of_Cases']
        pearson_corr = district_data_numeric.corr(method='pearson')['Number_of_Cases']
        kendall_corr = district_data_numeric.corr(method='kendall')['Number_of_Cases']

        partial_corr_series = pd.Series(index=district_data_numeric.columns)
        for col in district_data_numeric.columns:
            if col != 'Number_of_Cases':
                partial_corr_series[col] = partial_corr(data=district_data_numeric, x='Number_of_Cases', y=col, covar=[c for c in district_data_numeric.columns if c not in ['Number_of_Cases', col]]).iloc[0]['r']
            else:
                partial_corr_series[col] = 1.0

        distance_corr = district_data_numeric.apply(lambda x: distance_correlation(district_data_numeric['Number_of_Cases'], x))

        # Store correlations in dictionaries
        spearman_corr_dict[district] = spearman_corr.drop('Number_of_Cases')
        pearson_corr_dict[district] = pearson_corr.drop('Number_of_Cases')
        kendall_corr_dict[district] = kendall_corr.drop('Number_of_Cases')
        partial_corr_dict[district] = partial_corr_series.drop('Number_of_Cases')
        distance_corr_dict[district] = distance_corr.drop('Number_of_Cases')

    # Convert dictionaries to DataFrames
    spearman_corr_df = pd.DataFrame(spearman_corr_dict)
    pearson_corr_df = pd.DataFrame(pearson_corr_dict)
    kendall_corr_df = pd.DataFrame(kendall_corr_dict)
    partial_corr_df = pd.DataFrame(partial_corr_dict)
    distance_corr_df = pd.DataFrame(distance_corr_dict)

    # Select top 5 variables based on absolute correlation values for each method
    top_variables = {
        'Spearman': spearman_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Pearson': pearson_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Kendall': kendall_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Partial': partial_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist(),
        'Distance': distance_corr_df.abs().mean(axis=1).sort_values(ascending=False).head(5).index.tolist()
    }

    print("Top 5 variables for feature engineering based on correlation methods:")
    for method, variables in top_variables.items():
        print(f"{method}: {variables}")

    # Plotting the heatmap for Spearman's Rank Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(spearman_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Spearman Rank Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Pearson's Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(pearson_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Pearson Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Kendall's Tau Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(kendall_corr_df.T, annot=True, cmap='coolwarm', center=0)
    plt.title('Kendall Tau Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Partial Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(partial_corr_df.T.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title('Partial Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Plotting the heatmap for Distance Correlation
    plt.figure(figsize=(10, 8))
    sns.heatmap(distance_corr_df.T.astype(float), annot=True, cmap='coolwarm', center=0)
    plt.title('Distance Correlation Heatmap for All Districts on Number_of_Cases')
    plt.xlabel('Variables')
    plt.ylabel('Districts')
    plt.show()

    # Prepare data for the comparison line chart for top 5 variables
    top_variables_data = []
    for method, variables in top_variables.items():
        for variable in variables:
            top_variables_data.append({
                'Method': method,
                'Variable': variable,
                'Correlation': spearman_corr_df.loc[variable].mean() if method == 'Spearman' else
                               pearson_corr_df.loc[variable].mean() if method == 'Pearson' else
                               kendall_corr_df.loc[variable].mean() if method == 'Kendall' else
                               partial_corr_df.loc[variable].mean() if method == 'Partial' else
                               distance_corr_df.loc[variable].mean()
            })

    top_variables_df = pd.DataFrame(top_variables_data)

    # Plotting the comparison line chart for top 5 variables
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=top_variables_df, x='Variable', y='Correlation', hue='Method', marker='o')
    plt.title('Comparison of Top 5 Variables for Spearman, Pearson, Kendall, Partial, and Distance Correlations with respect to Number_of_Cases varaibale')
    plt.xlabel('Variables')
    plt.ylabel('Correlation')
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Call the function to calculate and plot correlations for all districts
calculate_and_plot_correlations(data)



def generate_insightful_report(data):
    unique_districts = data['District'].unique()
    report = []

    for district in unique_districts:
        district_data = data[data['District'] == district]
        columns_to_drop = ['District', 'Month', 'Year', 'Week', 'Weather Code', 'Total Precipitation (mm)']
        district_data_numeric = district_data.drop(columns=columns_to_drop).select_dtypes(include=[np.number])

        # Calculate correlations
        spearman_corr = district_data_numeric.corr(method='spearman')['Number_of_Cases']
        pearson_corr = district_data_numeric.corr(method='pearson')['Number_of_Cases']
        kendall_corr = district_data_numeric.corr(method='kendall')['Number_of_Cases']

        partial_corr_series = pd.Series(index=district_data_numeric.columns)
        for col in district_data_numeric.columns:
            if col != 'Number_of_Cases':
                partial_corr_series[col] = partial_corr(data=district_data_numeric, x='Number_of_Cases', y=col, covar=[c for c in district_data_numeric.columns if c not in ['Number_of_Cases', col]]).iloc[0]['r']
            else:
                partial_corr_series[col] = 1.0

        distance_corr = district_data_numeric.apply(lambda x: distance_correlation(district_data_numeric['Number_of_Cases'], x))

        # Create a DataFrame to compare the correlations
        comparison_df = pd.DataFrame({
            'Spearman': spearman_corr.drop('Number_of_Cases'),
            'Pearson': pearson_corr.drop('Number_of_Cases'),
            'Kendall': kendall_corr.drop('Number_of_Cases'),
            'Partial': partial_corr_series.drop('Number_of_Cases'),
            'Distance': distance_corr.drop('Number_of_Cases')
        })

        # Generate report for the district
        report.append(f"Correlation Analysis Report for {district} District")
        report.append("=" * 50)
        report.append(f"Top 5 Variables with Highest Spearman Correlation with 'Number_of_Cases':\n{spearman_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Pearson Correlation with 'Number_of_Cases':\n{pearson_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Kendall Correlation with 'Number_of_Cases':\n{kendall_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Partial Correlation with 'Number_of_Cases':\n{partial_corr_series.drop('Number_of_Cases').nlargest(5)}")
        report.append(f"\nTop 5 Variables with Highest Distance Correlation with 'Number_of_Cases':\n{distance_corr.drop('Number_of_Cases').nlargest(5)}")
        report.append("\n")

        # Detailed insights
        report.append(f"Detailed Insights for {district} District:")
        report.append(f"The variables with the highest Spearman correlation with 'Number_of_Cases' indicate a monotonic relationship. This means that as one variable increases, the other variable tends to increase or decrease consistently.")
        report.append(f"The Pearson correlation highlights the linear relationships between 'Number_of_Cases' and other variables. High Pearson correlation values suggest that changes in these variables are linearly associated with changes in 'Number_of_Cases'.")
        report.append(f"Kendall correlation, similar to Spearman, measures the ordinal association between variables. High Kendall correlation values suggest a strong ordinal relationship.")
        report.append(f"Partial correlation values show the relationship between 'Number_of_Cases' and other variables while controlling for the effect of other variables. This helps in understanding the direct influence of each variable on 'Number_of_Cases'.")
        report.append(f"Distance correlation captures both linear and non-linear associations. High distance correlation values indicate strong associations, regardless of the nature of the relationship.")
        report.append("\n")

    # Print the report
    for line in report:
        print(line)

# Generate the report for all districts
generate_insightful_report(data)



Testing